In [12]:
!pip install dask_ml
!pip install rake-nltk
# only if you haven't already installed these
!pip install bertopic[all] sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import dask.dataframe as dd
import re
import numpy as np
from collections import Counter
import nltk

# Only need to run this once per environment
nltk.download('punkt')
nltk.download('stopwords')


# TF–IDF dependencies
from dask_ml.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# RAKE for key-phrases
from rake_nltk import Rake

# (Optional) summarization
from transformers import pipeline

# — your settings —
DD_PATH        = '../Step_3_analysis/top_100_parquet/10.parquet'  # adjust as needed
TOP_N_TERMS    = 20
TOP_N_PHRASES  = 20
SUMMARIZE      = True                        # set False to skip summarization


[nltk_data] Downloading package punkt to /home/rgmatr1x/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rgmatr1x/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# seed‐word themes
themes = {
    # Core moment‑to‑moment play
    'gameplay': [
        'gameplay', 'mechanics', 'tactical shooter', 'precision', 'aim',
        'headshot', 'spray‑control', 'burst‑fire', 'recoil', 'crosshair',
        'peek', 'counter‑strafe', 'movement', 'jump‑peek', 'clutch',
        'bomb plant', 'defuse', 'round', 'eco‑round', 'overtime'
    ],

    # Weapons, grenades, recoil patterns
    'weapons': [
        'weapon', 'gun', 'rifle', 'ak', 'm4', 'awp', 'pistol', 'deagle',
        'smg', 'shotgun', 'sniper', 'knife', 'grenade', 'flashbang',
        'smoke', 'molotov', 'he‑nade', 'incendiary', 'zeus', 'spray',
        'pull‑out time', 'reload'
    ],

    # Map design and call‑outs
    'maps': [
        'map', 'layout', 'bombsite', 'call‑out', 'rotation', 'angles',
        'cover', 'line‑up', 'utility spot', 'choke‑point', 'dust2',
        'mirage', 'inferno', 'nuke', 'overpass', 'ancient', 'vertigo',
        'office', 'train', 'cache'
    ],

    # Ranked play, esport angle, organised competition
    'competitive': [
        'competitive', 'matchmaking', 'rank', 'elo', 'premier', 'global elite',
        'silver', 'faceit', 'esport', 'tournament', 'major', 'league',
        'teamplay', 'strat', 'timeout', 'coach', 'demo review', 'practice'
    ],

    # Game economy, skins, trading and cases
    'economy & skins': [
        'economy', 'money', 'buy', 'force', 'save', 'full‑buy', 'bonus‑loss',
        'skin', 'knife skin', 'case', 'capsule', 'stattrak', 'souvenir',
        'sticker', 'trade‑up', 'market', 'auction', 'rarity', 'float value',
        'pattern', 'lootbox'
    ],

    # Anti‑cheat and integrity concerns
    'anti_cheat': [
        'cheater', 'cheat', 'hacker', 'hack', 'wallhack', 'aimbot', 'spinbot',
        'vac', 'vac ban', 'prime', 'overwatch', 'smurf', 'rage', 'backtracking',
        'triggerbot', 'report', 'banwave', 'trust factor'
    ],

    # Performance, networking and technical stability
    'performance': [
        'fps', 'frame rate', 'stutter', 'lag', 'ping', 'tickrate', 'sub‑tick',
        'server', 'hit‑reg', 'netcode', 'desync', 'packet loss', 'freeze',
        'crash', 'memory leak', 'loading time', 'update', 'patch', 'driver'
    ],

    # Visual fidelity and art direction
    'visuals': [
        'visuals', 'graphics', 'shader', 'lighting', 'smoke effect',
        'blood splatter', 'particle', 'texture', 'model', 'animation',
        'ui', 'hud', 'crosshair style', 'ray tracing', 'color', 'resolution',
        'fov', 'viewmodel'
    ],

    # Audio design and voice comms
    'audio': [
        'audio', 'sound', 'footstep', 'sound cue', 'directional',
        'occlusion', 'gunshot', 'reverb', 'bomb beep', 'defuse sound',
        'voice chat', 'callout', 'microphone', 'radio command', 'volume',
        'sound bug', 'muffle', 'mix'
    ],

    # Social experience and community features
    'community': [
        'community', 'friends', 'lobby', 'party', 'team‑mate', 'toxic',
        'grief', 'vote kick', 'chat', 'text chat', 'mute', 'spray logo',
        'workshop', 'community server', 'surf', 'bhop', 'mods', 'plugin',
        'custom map', 'training map'
    ]
}

def slugify(name):
    return name.lower().replace(' ', '_')


In [15]:
# read only needed cols (add 'voted_up' for proxy pos/neg)
df = dd.read_parquet(
    DD_PATH,
    columns=['review', 'votes_up', 'voted_up', 'review_language']
)

# filter to English reviews only
df = df[df['review_language'] == 'english']

# replace NaN/None, force to str, then lowercase
df['review_text'] = (
    df['review']
    .fillna('')           # replace pandas NaN
    .astype(str)          # convert any None or <NA> to string
    .str.lower()          # lowercase
)

# cache partitions for faster reuse
df = df.persist()


In [16]:
likes_df    = df[df['voted_up'] == True]
dislikes_df = df[df['voted_up'] == False]

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

def top_tfidf_terms_sample(ddf, N=TOP_N_TERMS, sample_size=50000, random_state=42):
    # 1) count total docs in this bucket
    total = ddf['review_text'].dropna().count().compute()
    # 2) compute fraction so we sample ~sample_size docs
    frac = min(1.0, sample_size / total)
    # 3) sample that fraction
    sample = (
        ddf['review_text']
        .dropna()
        .sample(frac=frac, random_state=random_state)
        .compute()
        .tolist()
    )
    # 4) fit a classical TfidfVectorizer on the sampled docs
    vect = TfidfVectorizer(
        max_features=N,
        stop_words='english',
        token_pattern=r'(?u)\b\w+\b'
    )
    X = vect.fit_transform(sample)  # (n_samples x N) sparse matrix
    # 5) return the top-N feature names
    return vect.get_feature_names_out()

# run for likes & dislikes
likes_top_terms    = top_tfidf_terms_sample(likes_df)
dislikes_top_terms = top_tfidf_terms_sample(dislikes_df)

print("Top-TFIDF terms (likes):", likes_top_terms)
print("Top-TFIDF terms (dislikes):", dislikes_top_terms)


Top-TFIDF terms (likes): ['16' '3' 'best' 'classic' 'counterstrike' 'cs' 'csgo' 'fps' 'fun' 'game'
 'games' 'good' 'just' 'like' 'old' 'play' 'played' 'playing' 'time'
 'years']
Top-TFIDF terms (dislikes): ['16' 'bad' 'better' 'buy' 'counterstrike' 'cs' 'csgo' 'dont' 'game'
 'good' 'just' 'like' 'old' 'people' 'play' 'playing' 'server' 'servers'
 'steam' 'time']


In [18]:
# Cell 7: Top n-grams for actionable insights
from sklearn.feature_extraction.text import CountVectorizer

def top_ngrams_counts(ddf, ngram_range=(1,2), top_k=TOP_N_PHRASES,
                      sample_size=50000, random_state=42):
    # sample a fraction so we get ~sample_size docs
    total = ddf.dropna().count().compute()
    frac = min(1.0, sample_size / total)
    sample = (
        ddf.dropna()
           .sample(frac=frac, random_state=random_state)
           .compute()
           .tolist()
    )
    # extract n-grams
    vect = CountVectorizer(stop_words='english', ngram_range=ngram_range)
    X    = vect.fit_transform(sample)
    sums = X.sum(axis=0)
    terms = vect.get_feature_names_out()
    # pair up term → count
    counts = [(terms[i], int(sums[0, i])) for i in range(len(terms))]
    # sort and take top_k
    return sorted(counts, key=lambda x: x[1], reverse=True)[:top_k]

# run on likes / dislikes
likes_top_ngrams    = top_ngrams_counts(likes_df['review_text'])
dislikes_top_ngrams = top_ngrams_counts(dislikes_df['review_text'])

print("Top n-grams (likes):", likes_top_ngrams)
print("Top n-grams (dislikes):", dislikes_top_ngrams)


Top n-grams (likes): [('game', 18505), ('best', 5587), ('good', 4234), ('old', 3747), ('play', 3154), ('cs', 2802), ('best game', 2402), ('16', 2365), ('counterstrike', 2356), ('fps', 2029), ('like', 2027), ('games', 2003), ('fun', 1949), ('time', 1937), ('playing', 1891), ('played', 1856), ('classic', 1851), ('just', 1838), ('years', 1484), ('csgo', 1474)]
Top n-grams (dislikes): [('game', 802), ('play', 231), ('like', 168), ('just', 160), ('servers', 153), ('cs', 143), ('dont', 129), ('good', 123), ('server', 120), ('time', 105), ('better', 99), ('counterstrike', 98), ('buy', 96), ('csgo', 86), ('old', 85), ('16', 84), ('people', 82), ('playing', 79), ('steam', 79), ('bad', 76)]


In [19]:
# Cell 8: Chunked bullet summarization for multiple insights
from transformers import pipeline

def chunk_text(doc, max_words=100):
    """Split `doc` into chunks of ~max_words tokens each."""
    words = doc.split()
    return [" ".join(words[i : i + max_words]) 
            for i in range(0, len(words), max_words)]

if SUMMARIZE:
    # initialize once
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    
    def summarize_to_bullets(blob, max_bullets=5):
        bullets = []
        for chunk in chunk_text(blob, max_words=100):
            prompt = "Summarize the following into concise bullet points:\n\n" + chunk
            out = summarizer(prompt,
                             max_length=60,
                             min_length=20,
                             do_sample=False)[0]['summary_text']
            # split on newlines or “•” if present
            lines = [line.strip(" •-–") 
                     for line in out.replace("•","\n").splitlines() 
                     if line.strip()]
            bullets.extend(lines)
            if len(bullets) >= max_bullets:
                break
        return bullets[:max_bullets]
    
    # build your blobs (from previous cells)
    likes_blob    = " ".join(term for term, _ in likes_top_ngrams)
    dislikes_blob = " ".join(term for term, _ in dislikes_top_ngrams)
    
    # get multiple bullet insights
    like_insights    = summarize_to_bullets(likes_blob,    max_bullets=30)
    dislike_insights = summarize_to_bullets(dislikes_blob, max_bullets=30)
    
    # print them nicely
    print("=== What players LIKE ===")
    for b in like_insights:
        print("-", b)
    print("\n=== What players DISLIKE ===")
    for b in dislike_insights:
        print("-", b)


Device set to use cuda:0
Your max_length is set to 60, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 60, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


=== What players LIKE ===
- The best game 16 counterstrike fps like games fun time playing played classic just years csgo. The best good old play cs best.

=== What players DISLIKE ===
- 's guide on how to get the most out of your game time. The guide includes a list of tips and tricks that can be used to improve your game play.
